In [2]:
import os
import pandas as pd
import re
import json
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt


# Wrangling

### GEN Files

In [4]:
# Directory where raw CSV files are stored
directory = "../jupyter_notebook/data_samples"
# Parsing date strings, ignoring any timezone information and converting them to datetime objects
date_parser = lambda x: pd.to_datetime(x[:22])
# List to hold all the dataframes
dataframes = []
dict_of_dfs = {}

# Iterate through all files in the directory
for filename in os.listdir(directory):

    if re.match(r'gen_[A-Z]{2}_[A-Z0-9]+\.csv', filename):

        # Read the CSV file
        df = pd.read_csv(os.path.join(directory, filename), converters={'EndTime': date_parser}).set_index('EndTime')
       
        # Extract country and energy type from filename
        _, country, energy_type = filename.split('_')
        energy_type = energy_type.replace('.csv', '') # Remove the file extension

        # Add country and energy type as new columns
        df['CountryCode'] = country
        df['EnergyTypeCode'] = energy_type
        
        #Dropeamos valores duplicados que no tienen AREA ID.
        df.dropna(subset=['AreaID'],inplace=True)
        
        #Dropeamos el StartTime que no usaremos
        df.drop(columns=['StartTime'],inplace=True)
        
        #Reesampleamos
        df=df.resample('30T').sum()
        
        #Cambio de dato
        df['AreaID']=df['AreaID'].astype(str)
        
        #A los no existentes le ponemos none
        #df.loc[df['AreaID']=='0','quantity']=None
        
        #Ponemos como columna al EndTime
        df.reset_index(inplace=True)
        
        # Calcula la diferencia de tiempo entre filas consecutivas en minutos
        df['TimeDifference'] = df['EndTime'].diff().dt.total_seconds() / 60

        # Divide 60 por la diferencia de tiempo y guarda el resultado en una nueva variable
        valor = 60 / df['TimeDifference'][1]
        
        # Creamos una nueva columna 'grupo' que representa grupos consecutivos cada dos registros
        df['grupo'] = (df.index // valor) + 1

        # Iteramos sobre cada grupo para imputar NaN con la media del grupo
        for grupo, data_grupo in df.groupby('grupo'):
            if data_grupo['quantity'].isnull().all():
                continue  
            
            # Calculamos la media del grupo excluyendo NaN
            media_grupo = data_grupo['quantity'].mean(skipna=True)

            # Imputamos NaN con la media del grupo
            df.loc[df['grupo'] == grupo, 'quantity'] = df.loc[df['grupo'] == grupo, 'quantity'].fillna(media_grupo)
        
        #Dropeamos duplicados luego de imputar.
        df.dropna(inplace=True)
        
        #For resampling
        df.set_index('EndTime',inplace=True)
        
        numeric_cols = df.select_dtypes(include=['number'])
        categorical_cols = df.select_dtypes(exclude=['number', 'datetime64[ns]', 'bool'])
        
        # Resample the numeric columns and sum
        resampled_df_num = numeric_cols.resample('H').sum()

        # Resample the categorical columns.
        # Here, we take the first value. Adjust the method if needed (e.g., 'last', or a custom function to get the mode)
        resampled_df_cat= categorical_cols.resample('H').last()

        # Combine the resampled DataFrames back together
        resampled_df = pd.concat([resampled_df_num, resampled_df_cat], axis=1)
        
        #Dropeamos vacios luego de resamplear.
        resampled_df.dropna(inplace=True)
        
        # Append the dataframe to the list
        #dataframes.append(resampled_df)
        
        dict_of_dfs[f'{country}_{energy_type}']=resampled_df

# Concatenate all dataframes (if needed)
#final_df = pd.concat(dataframes)

In [8]:
dict_of_dfs['DE_B01']

,quantity,TimeDifference,grupo,AreaID,UnitName,PsrType,CountryCode,EnergyTypeCode
EndTime,,,,,,,,
2022-01-01 00:00:00+00:00,8642,30.0,1.0,10Y1001A1001A83F10Y1001A1001A83F,MAWMAW,B01B01,DEDE,B01B01
2022-01-01 01:00:00+00:00,17316,60.0,4.0,10Y1001A1001A83F10Y1001A1001A83F,MAWMAW,B01B01,DEDE,B01B01
2022-01-01 02:00:00+00:00,17342,60.0,6.0,10Y1001A1001A83F10Y1001A1001A83F,MAWMAW,B01B01,DEDE,B01B01
2022-01-01 03:00:00+00:00,17350,60.0,8.0,10Y1001A1001A83F10Y1001A1001A83F,MAWMAW,B01B01,DEDE,B01B01
2022-01-01 04:00:00+00:00,17375,60.0,10.0,10Y1001A1001A83F10Y1001A1001A83F,MAWMAW,B01B01,DEDE,B01B01
...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00+00:00,18513,60.0,17512.0,10Y1001A1001A83F10Y1001A1001A83F,MAWMAW,B01B01,DEDE,B01B01
2022-12-31 20:00:00+00:00,18121,60.0,17514.0,10Y1001A1001A83F10Y1001A1001A83F,MAWMAW,B01B01,DEDE,B01B01
2022-12-31 21:00:00+00:00,17916,60.0,17516.0,10Y1001A1001A83F10Y1001A1001A83F,MAWMAW,B01B01,DEDE,B01B01


In [ ]:
paises = ["SP", "UK", "DE", "DK", "HU", "SE", "IT", "PO", "NE"]

new_dict_of_dfs = {}

for pais in paises:
    # Crear una nueva clave con el formato "dic_[pais]"
    nueva_clave = f"dic_{pais}"

    # Filtrar DataFrames que contienen el código de país en la clave
    dataframes_filtrados = {key: df for key, df in dict_of_dfs.items() if pais in key}

    # Asignar el nuevo diccionario a la nueva clave
    new_dict_of_dfs[nueva_clave] = dataframes_filtrados

In [ ]:
output={}
for name,dataf in new_dict_of_dfs.items():
    concat_df = pd.concat(dataf)
    output[name]=concat_df

In [ ]:
output_1={}
for name,dataf in output.items():
    suma_grupo = dataf.groupby('EndTime')['quantity'].sum().reset_index()
    output_1[name]=suma_grupo

In [ ]:
fig, axs = plt.subplots(len(output_1), 1, figsize=(10, 6 * len(output_1)))

# Iterar sobre cada DataFrame en el diccionario
for i, (nombre, df) in enumerate(output_1.items()):
    axs[i].plot(df['EndTime'], df['quantity'], label=nombre)
    axs[i].set_title(nombre)
    axs[i].set_xlabel('EndTime')
    axs[i].set_ylabel('quantity')
    axs[i].legend()

plt.tight_layout()
plt.show()

### Load Files

In [ ]:
# Directory where raw CSV files are stored
directory = "../jupyter_notebook/data_samples"
# Parsing date strings, ignoring any timezone information and converting them to datetime objects
date_parser = lambda x: pd.to_datetime(x[:22])
# List to hold all the dataframes
dataframes = []
dict_of_dfs_load = {}

# Iterate through all files in the directory
for filename in os.listdir(directory):

    if re.match(r'load_[A-Z]{2}+\.csv', filename):
        
        # Read the CSV file
        df = pd.read_csv(os.path.join(directory, filename), converters={'EndTime': date_parser}).set_index('EndTime')
        
        _, country = filename.split('_')
        
        country = country.replace('.csv', '') # Remove the file extension
        
        country = 'dic_'+country
        
        dict_of_dfs_load[country]=df

In [ ]:
dict_of_dfs_load['dic_SP']

In [ ]:
fig, axs = plt.subplots(len(dict_of_dfs_load), 1, figsize=(10, 6 * len(dict_of_dfs_load)))

# Iterar sobre cada DataFrame en el diccionario
for i, (nombre, df) in enumerate(dict_of_dfs_load.items()):
    axs[i].plot(df.index, df['Load'], label=nombre)
    axs[i].set_title(nombre)
    axs[i].set_xlabel('EndTime')
    axs[i].set_ylabel('quantity')
    axs[i].legend()

plt.tight_layout()
plt.show()

## GEN and Load Files

In [ ]:
dict_gen=output_1.copy() #Dic Dataframes with gen energy
dict_load=dict_of_dfs_load #Dic Dataframes with load energy

# ARIMA Model

In [ ]:
p = 1  # replace with actual
d = 1  # replace with actual
q = 1  # replace with actual

# Fit the ARIMA model (using the known part of the time series)
model = ARIMA(df['quantity'], order=(p, d, q))
model_fit = model.fit()

# Forecast the missing values
# The 'steps' argument would be the number of hours in the missing months
forecast = model_fit.get_forecast(steps=4350) #hours in a monyh

# The forecast object contains the predicted values and other information
forecasted_values = forecast.predicted_mean

# You may also want to extract the confidence intervals of the forecasts
conf_int = forecast.conf_int()

To choose appropriate parameters for an ARIMA model, you can follow these steps:

1. Stationarity Check
First, check if your time series is stationary. This can be done using a statistical test, such as the Augmented Dickey-Fuller (ADF) test.

2. Differencing
If the time series is not stationary, apply differencing (d parameter in ARIMA) to make it stationary.

3. Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF)
Once you have a stationary series, plot the ACF and PACF. These plots will help you determine the p and q parameters.

ACF: Helps to identify the Moving Average (MA) component, which corresponds to q.
PACF: Helps to identify the Autoregressive (AR) component, which corresponds to p.
4. Model Identification
Based on the ACF and PACF plots, you can identify initial values for p and q:

The lag where the PACF cuts off is the estimated p.
The lag where the ACF cuts off is the estimated q.
5. Iterative Process
Model selection is an iterative process. You may need to try different combinations of p, d, and q and use model selection criteria like AIC (Akaike Information Criterion) to find the best model.

In [ ]:
# Load your data
# Assuming 'df' is your DataFrame with 'quantity' after loading your data

# Check for stationarity
result = adfuller(df['quantity'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
# If p-value > 0.05, the series is not stationary and you need to difference it

# Apply differencing if needed and determine 'd'
if result[1] > 0.05:
    # Assuming one level of differencing makes the series stationary
    d = 1
    df['quantity_diff'] = df['quantity'].diff(periods=d).dropna()
else:
    d = 0
    df['quantity_diff'] = df['quantity']

# Plot ACF and PACF
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(df['quantity_diff'], lags=40, ax=ax1)  # Change lags as needed
plot_pacf(df['quantity_diff'], lags=40, ax=ax2)  # Change lags as needed
plt.show()

# Look at where the plots cross the significance level for the first time to estimate 'p' and 'q'
# These are initial estimates and might need to be refined

# For example, if PACF cuts off after lag 2, and ACF tails off slowly, you might start with:
p = 2
q = 2

# Try fitting an ARIMA model with the initial parameters
model = sm.tsa.arima.ARIMA(df['quantity'], order=(p, d, q))
model_fit = model.fit()

# Evaluate model - check the AIC and the residual plots
print(model_fit.summary())
model_fit.plot_diagnostics(figsize=(15, 8))
plt.show()

# You may need to refine your (p, d, q) based on the diagnostics and AIC values